## Part III:  Loading Evaluation Labels

In [ ]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ candidates snorkel.db');

In [ ]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Loading the Training `CandidateSet`

In [ ]:
from snorkel.models import CandidateSet
candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

In [ ]:
import os
from hardware_utils import load_hardware_labels

label_set_name = 'Hardware Training Candidates -- Silver'
annotation_key_name = 'Hardware Training Labels -- Silver' 

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
%time gold_candidates, annotation_key = load_hardware_labels(\
                           session,\
                           label_set_name, \
                           annotation_key_name, \
                           candidates, \
                           filename, \
                           gold_attrib='stg_temp_min')
print "%d/%d Candidates have positive Labels" % (len(gold_candidates), len(candidates))

In [ ]:
from snorkel.models import Label

train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Silver').one()
print "%d/%d Candidates have positive Labels" % (len(train_gold), len(train))
print "%d Labels loaded" % session.query(Label).filter(
    Label.key == annotation_key).count()

In [ ]:
for set_name in ['Development']:
    candidate_set_name = 'Hardware %s Candidates' % set_name
    candidates = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name).one()
    label_set_name = 'Hardware %s Candidates -- Silver' % set_name
    annotation_key_name = 'Hardware %s Labels -- Silver' % set_name
    %time gold_candidates, annotation_key = load_hardware_labels(session,\
                           label_set_name, \
                           annotation_key_name, \
                           candidates, \
                           filename, \
                           gold_attrib='stg_temp_min')
    candidates_gold = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name + ' -- Silver').one()
    print "%d/%d Candidates in %s have positive Labels" % (
        len(candidates_gold), len(candidates), candidates)
    print "%d Labels loaded" % session.query(Label).filter(
        Label.key == annotation_key).count()

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ labels');

Next, in Part 4, we will auto-generate features for the `Candidates`